# PPI-network

This code was developed to find driver genes using a network-based approach on PPI (protein-protein interaction) data from the STRING database. Genes associated with bladder cancer were used as seed genes and provide a starting point for the analysis.

## Downloading the PPI network

Firstly, the PPI network must be downloaded from the STRING database.

The data is then saved as a `networkx` graph.

In [1]:
import requests, zlib

# download the file
zip_file_url = 'https://stringdb-static.org/download/protein.links.v10.5/9606.protein.links.v10.5.txt.gz'
STRING_response = requests.get(zip_file_url)

# decompress the file. ('data' is a string.)
data = zlib.decompress(STRING_response.content, zlib.MAX_WBITS|32)


## Processing the network data

The data is then processed. This means creating a `networkx` `Graph`, and using each line as an edge. Each of the first two 'words' of each edge represent the edge's nodes.

In [9]:
import networkx as nx

data_processed = data.split('\n')
del data_processed[0]
del data_processed[-1]

G = nx.Graph()
for item in data_processed:
    nodes = item.split(' ')
    G.add_edge(nodes[0],nodes[1],weight=int(nodes[2]))

print G.number_of_edges()
print G.number_of_nodes()

5676528
19576


## Pruning the network

#### Remove low confidence edges

In [10]:
to_delete_edges = []
for edge in G.edges():
    if G.get_edge_data(*edge)['weight'] < 400:
        to_delete_edges.append(edge)

for item in to_delete_edges:
    if G.has_edge(*item):
        G.remove_edge(*item)

to_delete_nodes = []
for key in G:
    if len(G[key])==0:
        to_delete_nodes.append(key)
        
for item in to_delete_nodes:
    G.remove_node(item)
    
print(G.number_of_nodes())
print(G.number_of_edges())

18838
792901


#### Remove all unnconnected nodes

In [11]:
largest_cc = max(nx.connected_components(G), key=len)

to_delete_unnconnected = []
for key in G:
    if key not in largest_cc:
        to_delete_unnconnected.append(key)

for node in to_delete_unnconnected:
    G.remove_node(node)
    
print(G.number_of_nodes())

18813


#### Remove nodes with no edges

In [12]:
to_delete_no_edges = []
for key in G:
    if len(G[key]) == 0:
        to_delete_no_edges.append(key)
        
print(len(to_delete_no_edges))

0


## Extend the seed genes using DIAMOnD

#### Import the seed genes

In [13]:
seed_file = 'seed_genes.tsv'
seed_genes = set()
for line in open(seed_file,'r'):
    # the first column in the line will be interpreted as a seed gene:
    line_data = line.strip().split('\t')
    seed_gene = line_data[0]
    seed_genes.add(seed_gene)
    
print len(seed_genes)

40


#### Ensure all the seed genes are in the network

In [14]:
all_genes_in_network = set(G.nodes())
seed_genes = set(seed_genes)
disease_genes = seed_genes & all_genes_in_network

if len(disease_genes) != len(seed_genes):
    print "DIAMOnD(): ignoring %s of %s seed genes that are not in the network" %(
        len(seed_genes - all_genes_in_network), len(seed_genes))
    
print len(disease_genes) - len(seed_genes)

0


#### Run DIAMOnD

In [15]:
import DIAMOnD

max_number_of_added_nodes = 160
alpha = 1

added_nodes = DIAMOnD.diamond_iteration_of_first_X_nodes(G,disease_genes,max_number_of_added_nodes,alpha)
added_nodes = [node[0] for node in added_nodes]

ext_seed_genes = added_nodes + list(seed_genes)
print len(ext_seed_genes)

200


## Edit edges linked to seed genes

#### Find edges linked to seed genes

In [18]:
edge_list = nx.edges(G,ext_seed_genes)

print(len(edge_list))

72232


#### Double the weights of edges connected to seed genes

In [22]:
for edge in edge_list:
    G[edge[0]][edge[1]]['weight'] *= 2

## Detect communities

In [1]:
import community

start = timer()
partition = community.best_partition(G,weight='weight',resolution=0.2)
end = timer()
print(end - start)

ImportError: No module named community

# Testing

In [23]:
counter = 0
for key in G:
    if counter is 0:
        print key
        print type(key)
    counter = counter + 1

9606.ENSP00000395733
<type 'str'>


In [25]:
G['9606.ENSP00000395733'][:][weight ]

TypeError: unhashable type

In [ ]:
counter = 0
for item in genes:
    if G.has_node(item):
        counter += 1
        
print(counter)

In [15]:
counter = 0
for edge in G.edges():
    if counter is 0:
        print edge
    counter = counter + 1
    
print G.edges['9606.ENSP00000395733','9606.ENSP00000332454']['weight']

('9606.ENSP00000395733', '9606.ENSP00000332454')
509


In [5]:
import community

ImportError: No module named community

In [4]:
import sys
print(sys.version)

help('modules')

2.7.13 (default, Apr  4 2017, 08:47:57) 
[GCC 4.2.1 Compatible Apple LLVM 8.1.0 (clang-802.0.38)]

Please wait a moment while I gather a list of all available modules...



/Users/clhastings/.virtualenvs/PPI-network/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


Audio_mac           _sqlite3            gensuitemodule      resource
BaseHTTPServer      _sre                gestalt             rexec
Bastion             _ssl                getopt              rfc822
CGIHTTPServer       _strptime           getpass             rlcompleter
Canvas              _struct             gettext             rmagic
Carbon              _symtable           glob                robotparser
CodeWarrior         _sysconfigdata      grp                 runpy
ColorPicker         _testcapi           gzip                scandir
ConfigParser        _threading_local    hashlib             sched
Cookie              _tkinter            heapq               scipy
Dialog              _warnings           hmac                select
DocXMLRPCServer     _weakref            hotshot             sets
EasyDialogs         _weakrefset         htmlentitydefs      setuptools
Explorer            abc                 htmllib             sgmllib
FileDialog          aepack              httplib   

/usr/local/Cellar/python/2.7.13/Frameworks/Python.framework/Versions/2.7/lib/python2.7/pkgutil.py:125: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  for item in walk_packages(path, name+'.', onerror):
